## q<sup>2</sup> Tutorial - 5. Interactivity with fancy plots

Whenever stellar parameters and chemical abundances are measured using equivalent widths (EWs), hundreds to thousands of EW measurements are made. Whether these are done automatically or by-hand, it is unavoidable to end up with a few bad measurements that could affect the results significantly. q<sup>2</sup> provides a couple of tools to facilitate the identification of these outliers.

Let's begin by importing q<sup>2</sup> and loading the input data from the Introduction; then, create the sun and arcturus star objects, attaching to them their corresponding input data and model atmospheres:

In [1]:
import q2
data = q2.Data('standards_stars.csv', 'standards_lines.csv')

sun = q2.Star('Sun')
sun.get_data_from(data)
sun.get_model_atmosphere()

arcturus = q2.Star('Arcturus')
arcturus.get_data_from(data)
arcturus.get_model_atmosphere()

For demonstration purposes, we are going to modify the equivalent width of the 5295.3 A line in Arcturus. This line appears first in the standards_lines.csv file, and it is therefore assigned the index 0 in the linelist attribute table. The measured value is 47.7 mA, but we are going to change it to 30.0 mA to simulate an incorrect measurement or a typo in the input data:

In [2]:
print(arcturus.linelist['wavelength'][0], arcturus.linelist['ew'][0])
arcturus.linelist['ew'][0] = 30.0

5295.3101 47.7


As we saw in the Introduction, the following command calculates the relative Arcturus - Sun iron abundance given the input parameters. We set the silent parameter to False to print the results on screen:

In [3]:
q2.specpars.iron_stats(arcturus, sun, silent=False)

FeI  : -0.602 +/- 0.056 (n= 37)
FeII : -0.545 +/- 0.029 (n=  9)


Note that the 1-sigma scatter of the line-by-line iron abundances from Fe I lines is significantly higher than that corresponding to the Fe II lines, presumably due to an outlier which might be somewhat difficult to find by inspecting a data table. In order to facilitate the identification of outliers, q<sup>2</sup> provides a function called "fancy_ironstats_plot", which takes as its only input parameter a star object:

In [4]:
q2.specpars.fancy_ironstats_plot(arcturus)

Loading BokehJS ...

The plot above is interactive. If the Hover tool is active on the right-hand-side menu, one can hover over each data point to examine its properties. You can also click-and-drag to move the plot around as long as the Pan tool is active. Box Zoom allows you to create a box of arbitrary size by clicking and dragging on a region of interest in the plot, and then zooms into it. Wheel Zoom can be used to zoom in and out of the plot with the mouse wheel. You can do this from within the plot or by axis. The latter allows you to zoom in and out one of the axes while keeping the other one constant; try scrolling the mouse wheel up or down with the mouse pointer on top of one of the axes labels. Click on the reset button to return to the default view.

A problem line is clearly identified in these plots at EP~4.4, REW~-5.25. Hovering reveals that this outlier is the 5295.3 A feature, which is the line that we modified manually above.

As we saw in Part 4 of this Tutorial, we can use the abundances.get_one function to calculate elemental abundances given a star object (arcturus) and a reference (sun). Let's use it to re-calculate Arcturus' iron abundances from Fe I and Fe II lines:

In [5]:
q2.abundances.get_one(arcturus, species_ids=['FeI', 'FeII'], Ref=sun, silent=False)

*** Begin FeI:
A(FeI)  =  6.829 +/- 0.061 (# of lines = 37)
[FeI/H] = -0.602 +/- 0.055 (# of lines = 37)
---FeI done
*** Begin FeII:
A(FeII)  =  6.867 +/- 0.072 (# of lines = 9)
[FeII/H] = -0.545 +/- 0.027 (# of lines = 9)
---FeII done
All species completed


The fancy_ironstats_plot function was created to identify outliers when using the specpars module. In other words, it is the function that you should use when determining stellar parameters. A similar tool exists to identify outliers in a generic elemental abundance calculation. While this example illustrates how to examine iron abundances, which can also be done with fancy_ironstats_plot, it is actually possible to examine any elemental abundance determined with the abundances.get_one function by using the abundances.fancy_abund_plot function, not only iron.

The arcturus object has attributes FeI and FeII attached to it from the calculation above. The command below allows us to examine interactively the abundances determined with the Fe I lines only. The second argument, 'FeI' in this example, is required because abundances.get_one attaches as many abundance attributes to the star object as requested by the species_ids argument:

In [6]:
q2.abundances.fancy_abund_plot(arcturus, species_id='FeI')

Loading BokehJS ...

Again, the 5295.3 A outlier line stands out and is easily identified with the tools provided in this interactive plot.

One can, of course, examine a similar plot for the Fe II lines only. If a star object has many more abundance attributes attached using the abundances.get_one function, you can select any one of them with the species_id argument.

If the star object has abundances determined differentially, the abundances.fancy_abund_plot will show two panels, as in this example, one with the absolute abundances (upper panel) and one with the relative ones (lower panel). If abundances are calculated without a reference, only the upper panel will be shown.

The fancy plot tools introduced here help identifying outliers. It is up to the user to decide what to do with them. It is recommended to re-examine the spectra and/or re-measure the problem spectral line(s), but one should be careful to maintain consistency with the rest of the analysis. Careful inspection of the problem spectral line(s) may even result in a decision to discard a few features, a decision that should be solely driven by a critical evaluation of the observational data set.

This concludes the q<sup>2</sup> Tutorial for determining MOOG-based spectroscopic parameters and chemical abundances. In a "bonus" Part 6 we will find out how to use q<sup>2</sup> to estimate fundamental parameters using theoretical grids of stellar models, or isochrones.